In [1]:
!pip install requests
!pip install pysolr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pysolr: filename=pysolr-3.10.0-py2.py3-none-any.whl size=20051 sha256=fdac2381340552e214546dd0914ea7a9d82ca8385cbbaff978673775c5693726
  Stored in directory: /root/.cache/pip/wheels/14/a7/f3/9ce88a9a83d46c520ad83ccecaf82b4566b00985abbf00283e
Successfully built pysolr


In [2]:

ngrok_url = "https://f10f-223-181-208-107.ngrok-free.app/solr/#/~cloud"

import requests
import pandas as pd
import json


In [7]:
!pip install requests
!pip install pysolr

ngrok_url = "https://f10f-223-181-208-107.ngrok-free.app/solr"

import requests
import pandas as pd
import json

def createCollection(p_collection_name):
    url = f"{ngrok_url}/admin/collections"
    params = {
        "action": "CREATE",
        "name": p_collection_name,
        "numShards": 1,
        "replicationFactor": 1
    }
    response = requests.get(url, params=params)
    return response.json()

def indexData(p_collection_name, p_exclude_column):

    file_path = '/content/Employee Sample Data 1.csv'
    try:
        employee_data = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        employee_data = pd.read_csv(file_path, encoding='ISO-8859-1')


    if p_exclude_column in employee_data.columns:
        employee_data = employee_data.drop(columns=[p_exclude_column])


    for index, row in employee_data.iterrows():
        document = row.to_dict()
        response = requests.post(f'{ngrok_url}/{p_collection_name}/update/json/docs',
                                 data=json.dumps(document),
                                 headers={'Content-Type': 'application/json'})
        if response.status_code == 200:
            print(f'Document indexed successfully: {document}')
        else:
            print(f'Failed to index document: {document}, Status code: {response.status_code}, Response: {response.text}')


def searchByColumn(p_collection_name, p_column_name, p_column_value):
    url = f'{ngrok_url}/{p_collection_name}/select'
    params = {
        'q': f'{p_column_name}:{p_column_value}',
        'wt': 'json'
    }
    response = requests.get(url, params=params)
    return response.json()

def getEmpCount(p_collection_name):
    url = f'{ngrok_url}/{p_collection_name}/select'
    params = {
        'q': '*:*',
        'rows': 0,
        'wt': 'json'
    }
    response = requests.get(url, params=params)
    return response.json()

def getDepFacet(p_collection_name):
    url = f'{ngrok_url}/{p_collection_name}/select'
    params = {
        'q': '*:*',
        'facet': 'true',
        'facet.field': 'Department',
        'wt': 'json'
    }
    response = requests.get(url, params=params)
    return response.json()

def delEmpById(p_collection_name, p_employee_id):
    url = f'{ngrok_url}/{p_collection_name}/update?commit=true'
    data = f'<delete><id>{p_employee_id}</id></delete>'
    headers = {'Content-Type': 'text/xml'}
    response = requests.post(url, data=data, headers=headers)


    print("Response text:", response.text)


    if response.status_code == 200:

        if '<int name="status">0</int>' in response.text:
            return "Delete operation successful."
        else:
            return "Delete operation failed."
    else:
        return f"Error: {response.status_code} - {response.text}"


In [8]:
v_nameCollection = 'Hash_Danish'
v_phoneCollection = 'Hash_5294'


createCollection(v_nameCollection)
createCollection(v_phoneCollection)

{'responseHeader': {'status': 400, 'QTime': 37},
 'error': {'metadata': ['error-class',
   'org.apache.solr.common.SolrException',
   'root-error-class',
   'org.apache.solr.common.SolrException'],
  'msg': 'collection already exists: Hash_5294',
  'code': 400}}

In [9]:
count_before_indexing = getEmpCount(v_nameCollection)
print("Employee count before indexing:", count_before_indexing)

indexData(v_nameCollection, 'Exit Date')
indexData(v_phoneCollection, 'Gender')

delete_response = delEmpById(v_nameCollection, 'E02002')
print("Delete response:", delete_response)

count_after_deletion = getEmpCount(v_nameCollection)
print("Employee count after deletion:", count_after_deletion)

search_results_dept = searchByColumn(v_nameCollection, 'Department', 'IT')
print("Search results by Department:", search_results_dept)

search_results_gender = searchByColumn(v_nameCollection, 'Gender', 'Male')
print("Search results by Gender:", search_results_gender)

department_facet = getDepFacet(v_nameCollection)
print("Department facet:", department_facet)


Employee count before indexing: {'responseHeader': {'zkConnected': True, 'status': 0, 'QTime': 5, 'params': {'q': '*:*', 'rows': '0', 'wt': 'json'}}, 'response': {'numFound': 7, 'start': 0, 'numFoundExact': True, 'docs': []}}
Document indexed successfully: {'Employee ID': 'E02002', 'Full Name': 'Kai Le', 'Job Title': 'Controls Engineer', 'Department': 'Engineering', 'Business Unit': 'Manufacturing', 'Gender': 'Male', 'Ethnicity': 'Asian', 'Age': 47.0, 'Hire Date': '2/5/2022', 'Annual Salary': '$92,368 ', 'Bonus %': '0%', 'Country': 'United States', 'City': 'Columbus'}
Document indexed successfully: {'Employee ID': 'E02003', 'Full Name': 'Robert Patel', 'Job Title': 'Analyst', 'Department': 'Sales', 'Business Unit': 'Corporate', 'Gender': 'Male', 'Ethnicity': 'Asian', 'Age': 58.0, 'Hire Date': '10/23/2013', 'Annual Salary': '$45,703 ', 'Bonus %': '0%', 'Country': 'United States', 'City': 'Chicago'}
Document indexed successfully: {'Employee ID': 'E02004', 'Full Name': 'Cameron Lo', 'Job 

In [11]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.3 MB/s eta 0:00:00


In [15]:

ngrok_url = "https://f10f-223-181-208-107.ngrok-free.app/solr"

In [16]:
import gradio as gr
import requests
import json


def fetch_collections():
    url = f"{ngrok_url}/admin/collections?action=LIST"
    response = requests.get(url)
    collections = response.json().get("collections", [])
    return collections
def searchByColumn(p_collection_name, p_column_name, p_column_value):
    url = f'{ngrok_url}/{p_collection_name}/select'
    params = {
        'q': f'{p_column_name}:{p_column_value}',
        'wt': 'json'
    }
    response = requests.get(url, params=params)
    return response.json()

def getEmpCount(p_collection_name):
    url = f'{ngrok_url}/{p_collection_name}/select'
    params = {
        'q': '*:*',
        'rows': 0,
        'wt': 'json'
    }
    response = requests.get(url, params=params)
    return response.json()

def delEmpById(p_collection_name, p_employee_id):
    url = f'{ngrok_url}/{p_collection_name}/update?commit=true'
    data = f'<delete><id>{p_employee_id}</id></delete>'
    headers = {'Content-Type': 'text/xml'}
    response = requests.post(url, data=data, headers=headers)

    if response.status_code == 200:
        if '<int name="status">0</int>' in response.text:
            return "Delete operation successful."
        else:
            return "Delete operation failed."
    else:
        return f"Error: {response.status_code}"

def getDepFacet(p_collection_name):
    url = f'{ngrok_url}/{p_collection_name}/select'
    params = {
        'q': '*:*',
        'facet': 'true',
        'facet.field': 'Department',
        'wt': 'json'
    }
    response = requests.get(url, params=params)
    return response.json()
def interface(collection_name, action, search_column=None, search_value=None, emp_id_to_delete=None):
    if action == "Search Employee":
        return searchByColumn(collection_name, search_column, search_value)
    elif action == "Get Employee Count":
        return getEmpCount(collection_name)
    elif action == "Delete Employee":
        return delEmpById(collection_name, emp_id_to_delete)
    elif action == "Get Department Facet":
        return getDepFacet(collection_name)
    else:
        return "Invalid action"

def collection_interface(selected_collection, action, search_column, search_value, emp_id_to_delete):
    result = interface(selected_collection, action, search_column, search_value, emp_id_to_delete)
    return result

def main_interface():
    collections = fetch_collections()

    with gr.Blocks() as demo:
        gr.Markdown("### Employee Management System with Solr")

        collection_dropdown = gr.Dropdown(label="Select Collection", choices=collections)
        action_dropdown = gr.Dropdown(label="Select Action", choices=["Search Employee", "Get Employee Count", "Delete Employee", "Get Department Facet"])

        search_column = gr.Textbox(label="Search Column (if applicable)", placeholder="Enter column name")
        search_value = gr.Textbox(label="Value to Search (if applicable)", placeholder="Enter value to search")
        emp_id_to_delete = gr.Textbox(label="Employee ID to Delete (if applicable)", placeholder="Enter Employee ID to delete")

        result_output = gr.JSON(label="Result")

        # Button to execute the action
        action_button = gr.Button("Execute Action")

        action_button.click(
            fn=collection_interface,
            inputs=[collection_dropdown, action_dropdown, search_column, search_value, emp_id_to_delete],
            outputs=result_output
        )

    return demo

# Launch the Gradio app
main_interface().launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://536ee082047a25fdb9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
